In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 40.0 MB/s eta 0:00:00


In [ ]:
!pip install keras_nlp
!pip install huggingface-hub
!pip install nltk
!pip install rouge-score
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.5/584.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 42.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=342bdd1f4163b2cbe76110e1200da9f9f05bef3c0409f118d74b24b817b00ec7
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00


In [ ]:
import os
import logging

import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from datasets import DatasetDict, Dataset, load_dataset

# Only log error messages
tf.get_logger().setLevel(logging.ERROR)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
# The percentage of the dataset you want to split as train and test
TRAIN_TEST_SPLIT = 0.1

MAX_INPUT_LENGTH = 1024  # Maximum length of the input to the model
MIN_TARGET_LENGTH = 5  # Minimum length of the output by the model
MAX_TARGET_LENGTH = 128  # Maximum length of the output by the model
BATCH_SIZE = 1 # Batch-size for training our model
LEARNING_RATE = 2e-5  # Learning-rate for training our model
MAX_EPOCHS = 1  # Maximum number of epochs we will train the model for
# MODEL_CHECKPOINT = "google/flan-t5-small"
MODEL_CHECKPOINT ="flax-community/t5-base-cnn-dm"

In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/298_B_final_Sem/Summarization_Data_Set/summarization_cleaned_file_Himanshee(9-14).csv')

In [ ]:
df.info()
# df.head(10)
# df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295503 entries, 0 to 295502
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   article     295503 non-null  object
 1   highlights  295503 non-null  object
dtypes: object(2)
memory usage: 4.5+ MB


In [ ]:
# Split the data into 70% train and 30% rest (which will be used for test and val)
train_df, rest_df = train_test_split(df, test_size=0.3, random_state=42)

# Split the rest into 50% test and 50% validation
test_df, val_df = train_test_split(rest_df, test_size=0.5, random_state=42)

In [ ]:
# limit=1000
# if limit!=-1:
#   test_df=test_df.head(limit)

In [ ]:
train = Dataset.from_pandas(train_df).rename_column("__index_level_0__", "id")
val = Dataset.from_pandas(val_df).rename_column("__index_level_0__", "id")
test = Dataset.from_pandas(test_df).rename_column("__index_level_0__", "id")
full_dataset = DatasetDict({'train': train, 'val': val, 'test': test})

In [ ]:
full_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 206852
    })
    val: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 44326
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 1000
    })
})

In [ ]:
# from transformers import AutoTokenizer, AutoModel

# model = TFAutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
if MODEL_CHECKPOINT in ["flax-community/t5-base-cnn-dm"]:
    prefix = "summarize: "
else:
    prefix = ""

In [ ]:
!pip install rouge

In [ ]:
# def generate_summary(input):
#     inputs =  tokenizer(input["article"], return_tensors="pt", padding=True, truncation=True, max_length=512)
#     outputs = model.generate(
#         input_ids=inputs["input_ids"],
#         attention_mask=inputs["attention_mask"],
#         max_length=120,
#         do_sample=False,
#         #top_p=0.9
#     )
#     summaries = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
#     return {"summary": summaries}
def generate_summary(batch):
    # Ensure padding and truncation
    inputs = tokenizer(batch["article"], return_tensors="tf", padding=True, truncation=True, max_length=512)
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=120,
        do_sample=False,
        #top_p=0.9
    )
    # Decode all generated summaries
    summaries = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return {"summary": summaries}

In [ ]:
!python --version

Python 3.10.12


In [ ]:
from rouge import Rouge
# print(type(test[0].items))
summary_dataset = test.map(generate_summary, batched=True, batch_size=10)

# # Extract generated and reference summaries
generated_summaries = summary_dataset["summary"]
reference_summaries = [item["highlights"] for item in test]


# Assuming test_set is your dataset with articles
# generated_summaries = []
# reference_summaries = []

# for sample in test:
#     print(sample)
#     article_text = sample["article"]
#     generated_summary = generate_summary(sample)

#     # Assuming you have reference summaries in the dataset under the key "highlights"
#     reference_summary = sample["highlights"]

#     generated_summaries.append(generated_summary)
#     reference_summaries.append(reference_summary)

# # # # Evaluate the entire generated summaries against the entire reference summaries
# generated_summaries_concatenated = " ".join(generated_summaries)
# reference_summaries_concatenated = " ".join(reference_summaries)


/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset, load_metric

metric = load_metric("rouge")

<ipython-input-21-faaac16489c9>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [ ]:
# Now compute ROUGE scores
scores = metric.compute(predictions=generated_summaries, references=reference_summaries)

# Output the scores
print(scores)
rouge_l_score = scores["rougeL"].mid.fmeasure
rouge_n1_score = scores["rouge1"].mid.fmeasure
rouge_n2_score = scores["rouge2"].mid.fmeasure

{'rouge1': AggregateScore(low=Score(precision=0.22727941565792056, recall=0.35601457754178223, fmeasure=0.26648670112905437), mid=Score(precision=0.234458324172363, recall=0.3663210063757626, fmeasure=0.2739649119705182), high=Score(precision=0.24200781782109104, recall=0.37619657077633634, fmeasure=0.2811891292787973)), 'rouge2': AggregateScore(low=Score(precision=0.07765405791992806, recall=0.12333696374467089, fmeasure=0.09129422045649963), mid=Score(precision=0.08300591112572184, recall=0.1313303078840574, fmeasure=0.09733211171096168), high=Score(precision=0.0885292127154849, recall=0.13970052285375592, fmeasure=0.10352211601152879)), 'rougeL': AggregateScore(low=Score(precision=0.15499196000873608, recall=0.24692867173517782, fmeasure=0.1827254392274835), mid=Score(precision=0.1607602326505987, recall=0.2555189523745637, fmeasure=0.1889235433463407), high=Score(precision=0.16660518536302446, recall=0.26463792612997483, fmeasure=0.19524962122737521)), 'rougeLsum': AggregateScore(l

In [ ]:
print(f"RougeL: {rouge_l_score:.4f} - RougeN1: {rouge_n1_score:.4f} - RougeN2: {rouge_n2_score:.4f}")


RougeL: 0.1889 - RougeN1: 0.2740 - RougeN2: 0.0973
